In [2]:
#Importing Necessary Libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')
import os
import sys
import logging
import argparse
import joblib
from datetime import datetime
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline   
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor


Exploring the Data


In [3]:
df = pd.read_csv('preprocessed_content.csv')
df.head(10)

,Unnamed: 0,filename,ticker,year,preprocessed_content,ner_entities,e_score,s_score,g_score,total_score
0,0,ASX_BSX_2020.pdf,BSX,2020,style guide colour colour use imagecolour prof...,"['bk%', 'rgb', 'un', 'el ectric mine consortiu...",3.16,18.00,11.83,32.98
1,1,ASX_BSX_2022.pdf,BSX,2022,sustainability report look mining green office...,"['murray street', 'west perth', 'west perth', ...",2.83,12.86,10.32,26.02
2,2,ASX_EXR_2022.pdf,EXR,2022,report environment social governance esg basel...,"['september', 'mongolia', 'australia', 'austra...",3.81,4.28,5.86,13.94
3,3,LSE_ADM_2019.pdf,ADM,2019,corporate social responsibilty report introduc...,"['david stevens', 'csr board', 'just over yea...",16.38,14.20,5.90,36.36
4,4,LSE_ADM_2020.pdf,ADM,2020,sustainability admiral commit maintain respons...,"['year', 'health & wellbeing', 'a -month', 'on...",15.89,13.51,5.38,34.78
5,5,LSE_ADM_2021.pdf,ADM,2021,look future sustainability report customer cle...,"['annual', 'admiralgroup.co.uk', 'this year', ...",15.91,13.49,4.85,34.25
6,6,LSE_BBY_2020.pdf,BBY,2020,sustainability report guidance version content...,"['globa l sustainability', 'glossary & abbrevi...",2.11,5.20,4.60,11.92
7,7,NASDAQ_AAPL_2019.pdf,AAPL,2019,apple develop new alloy enable use percent rec...,"['apple', 'fiscal year', '•', 'lisa jackson', ...",0.55,13.58,10.47,24.32
8,8,NASDAQ_AAPL_2021.pdf,AAPL,2021,report apple report content introduction etter...,"['tim cook', 'marter chemistry', '︎', 'apple p...",0.18,7.69,8.86,16.72
9,9,NASDAQ_AAPL_2022.pdf,AAPL,2022,reportenvironmental introduction letter cook r...,"['apple', 'tim cook', 'united nations sustaina...",0.65,6.86,8.95,16.45


In [4]:
df.tail(20)

,Unnamed: 0,filename,ticker,year,preprocessed_content,ner_entities,e_score,s_score,g_score,total_score
846,846,NYSE_XYL_2020.pdf,XYL,2020,solve water resilient world sustainability rep...,"['patrick decker', 'president & ceo', 'annual'...",2.94,9.56,5.37,17.86
847,847,NYSE_XYL_2021.pdf,XYL,2021,report message message chief people sustainabi...,"['patrick decker', 'president & ceo ', 'inc...",3.53,7.81,5.11,16.45
848,848,NYSE_YUM_2019.pdf,YUM,2019,global citizenship sustainability reportunlock...,"['people culture &', 'inclusion & belonging', ...",4.10,10.86,5.39,20.36
849,849,NYSE_YUM_2020.pdf,YUM,2020,purpose feed future global citizenship sustain...,"['covid- response ethics & human', 'a year', '...",3.35,10.55,4.58,18.48
850,850,NYSE_YUM_2021.pdf,YUM,2021,global citizenship sustainability purpose feed...,"['•', 'the last two years', 'one', 'year two',...",3.82,12.39,4.50,20.72
851,851,NYSE_YUM_2022.pdf,YUM,2022,brand global citizenship sustainability report...,"['un', 'communities & society', 'four', 'more ...",4.25,12.13,4.74,21.12
852,852,NYSE_ZBH_2019.pdf,ZBH,2019,environmental social governance sustainability...,"['warsaw', 'indiana', 'millions', 'six', 'ems'...",3.80,18.20,12.41,34.25
853,853,NYSE_ZBH_2020.pdf,ZBH,2020,sustainability report corporate overview missi...,"['health & safety', 'communication & education...",4.30,18.72,11.08,34.10
854,854,NYSE_ZBH_2021.pdf,ZBH,2021,sustainability environmental social governance...,"['health & safety', 'communication & education...",4.74,17.84,10.35,32.94
855,855,NYSE_ZTS_2020.pdf,ZTS,2020,journey well world introduction letter ceo rep...,"['three', 'the past year', 'first', 'march', '...",2.72,13.98,11.49,28.19
